# HEY, ISA! 

05.21 Consider normalizing next sesh (see notes in solver). I'm going to sleep now. 

Missing: Boundary condition for dust particle! 

In [1]:
import math
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt 
import sys, os 
import time
import csv
#from decimal import *
%matplotlib inline

### Inputs 

Variable types: 
    - Constants
    - Simulation inputs
    - Plasma parameters 
    - Domain parameters 
    - Specific weight 
    - Particle arrays (position and velocity) 

In [2]:
# Constants 
EPS0 = 8.85 * 10 ** -12 
QE = 1.602 * 10 ** -19 # C 
AMU = 1.661 * 10 ** -27 # kg 
MI = 40 * AMU # mass of Ar+ ion 
Q = QE / EPS0 

# Simulation inputs 

N = 20000 # max particles (buffer)
n0 = 1 * 10 ** 12 # density 
V_ref = 0 # reference potential 
Te = 1  # electron temperature in eV 
Ti = 0.1 # ion temperature in eV
V_d = -5.1 # dust potential (use values from lit)
v_drift = 7000 
tol = 1 * 10 ** -4
N_ts = 1 # number of time steps 


# Plasma parameters 
lambd_D = np.sqrt( EPS0 * Te /(n0 * QE))   
vth = np.sqrt((2 * QE * Ti) / MI) 
v0 = 0.2 # stream velocity 
## vth = np.sqrt(Ti/MI) creates tachyons!


# Domain parameters 
nx = 10
ny = 10
J = nx * ny 
dx = dy = dh = lambd_D
Lx = nx * dx 
Ly = ny * dy 
dt = (0.1 * dx) / v_drift 
np_in = (ny-1) * 15  

# Specific weight 
flux = n0 * v_drift * Ly 
npt = flux * dt # particles created per timestep 
sw = npt / np_in # specific weight 
q_mp = 1 # macroparticle charge 

# Particle arrays
p_pos = np.zeros([N,2])
p_velo = np.zeros([N,2])

print('Debye length = %f m' %lambd_D, 
      'domain length (x and y dir) = %f m' %Lx,
      'thermal velocity = %f m/s' %vth,
      'drift velocty = %d m/s' %v_drift,
      'time step = %e s' %dt, 
      'particles inserted per cell = %d' %np_in, 
      'flux = %e' %flux, 
      'particles created per time step = %e' %npt, 
      'specific weight = %e' %sw, 
      'number of cells = %d' %J,
      sep="\n")

Debye length = 0.007433 m
domain length (x and y dir) = 0.074326 m
thermal velocity = 694.434745 m/s
drift velocty = 7000 m/s
time step = 1.061799e-07 s
particles inserted per cell = 135
flux = 5.202815e+14
particles created per time step = 5.524345e+07
specific weight = 4.092107e+05
number of cells = 100


### 2D potential solver 
#### Uses Gauss-Seidel method (Ax = b) 

Where: 

- M (stencil matrix) is A 
- V (potenital) is x 
- b (boltzmann relation) is b 

Thus we have, 

\begin{equation*}
MV = b,  
\end{equation*}

where M is a square array, and V, b are column vectors. 

We solve for V using forward substitution, shown below. 

\begin{equation*}
V = \frac{1}{M_{ii}} [ b_i - \sum_{j=1}^{i-1} M_{ij} V_j - \sum_{j=1+1}^n M_{ij} V_j^k ].
\end{equation*} 
 

Notes: 

There are appears to be operational issues with this solver. It may have something to do with the fact that python, unlike matlab, does not work with v. large/small numbers well. 

In [3]:
start = time.clock() # Keeping track of computational time 

def solver_2d(rho, tol, Te, n0, V, Q, M): 
    'Uses Gauss Sidel method Ax = b'
    # rho = charge density (array)  
    #tol = tolerance 
    # Te = electron temp in eV 
    # n0 = electron density 
    # V = potential 
    # Q = QE/EPS0 
    # M = stencil matrix 
    
    iterations = 1
        
    NX = np.size(rho,0)
    NY = np.size(rho,1)
    NN = np.size(rho)
    
    ni = np.reshape(rho, (NN,1)) # ion density  
    V = np.reshape(V, (np.size(V), 1))  
    ## reshaping to column vectors for boltzmann relation calculation

    # print(np.size(x), np.size(b0), np.size(M)) 
    # 100 100 10000 as of 5/22
    
    # print(NX, NY, NN) 
    # 10 10 100 as of 5/22
    
    for i in range (0, iterations):

        # Boltzmann relation for electrons (fluid teatment)
        
        #b = 0 
        
        b = ni - (n0 * np.exp((V) / Te))
        ## initial values: n0 = 1E12, b0 = zeros(NN,1), x = zeros(V, 1)
        b = b * (-1 * Q)
        #b = np.nan_to_num(b)
        ## print(b) <-- careful, this can crash your program!
        
        # Boundaries 
        
        #b[0:NX-1] = 0 
        #b[NN-NX+1 : NN] = 0 
        #b[NX: NX : NN] = 0 
        #b[1:NX:NN] = V_ref
        
        for i in range (0, NN):
            V[i] =  (b[i] - np.dot(M[i, 0:i], V[0: i]) -  
                    np.dot(M[i, i+1 : NN-1] , V[i+1:NN-1])) / M[i,i]
            
            #V = np.nan_to_num(V)

        if count%100 == 0 or count == iter:            
            R = np.linalg.norm(b - M * V) 
            if R <= tol:
                print('Solver converged.')
                break
            #elif R > tol: 
                #print('Solver failed to converge. Check R!')

    V = np.reshape(V, (NX,NY))
    return V
print("Clocking in at %s seconds"  % (time.clock() - start))

Clocking in at 0.0004459999999997244 seconds


### Main Cycle

- Creates stencil matrix for potential solver 
- Solves E field 
- Generates and distributes new particles 
- Moves particles 
- Add/removes particles at boundaries (not fully implemented) 

Notes: 5.21.2018

Plot works (phase space), and particles are randomly scattered in each time step. However it does not appear that any appreciable number of particles are being generated (I dont notice a difference between plots). May have have to do with the fact that the solver technically doesn't work...

Notes: 5.22.2018 

Contour plot for V doesn't work cause nans aren't useful

In [4]:
start = time.clock()

# Stencil matrix 

M = np.zeros((J,J))

for i in range (1, nx-1):
    for j in range(1, ny-1): 
        u = (j-1) * nx + i
        M[u, u] = -4 / (dh ** 2) 
        M[u, u-1] = 1 / (dh ** 2)
        M[u, u+1] = 1 / (dh ** 2)
        M[u, u-nx] = 1 / (dh ** 2)
        M[u, u+ (nx-2)] = 1 / (dh ** 2)    

for i in range (0, nx):
    u = i+1
    M[u,u] = -1 / dh
    M[u,u+nx] = 1 / dh 
    
for i in range (0,nx): 
    u = (ny-1) * nx  + i 
    M[u, u-nx] = 1 / dh
    M[u, u] = -1 / dh 
    
for j in range (0, ny): 
    u = (j-1) * nx + nx 
    M[u, :] = np.zeros([1, J])
    M[u, u-1] = 1 / dh 
    M[u, u] = -1 / dh 
    
for j in range(0, ny): 
    u = (j-1) * nx + 1 
    M[u, :] = np.zeros([1, J])
    M[u, u] = 1   

#np.set_printoptions(threshold=np.nan)
#print(M)

# Initialize
NP = 0
V = np.ones((ny,nx)) * V_ref

# Solve, move, plot  

print('Calculating...')

for count in range(0, N_ts):
    
    q = np.zeros((nx, ny))
    rho = np.zeros((ny, nx))
    Ex = np.zeros([nx, ny])
    Ey = np.zeros([nx, ny])
     
    for p in range (1, NP):  
        
        fi = 1 + (p_pos[p,0] / (dh)) 
        i = np.floor(fi)  
        hx = fi - i  
        
        fj = 1 + (p_pos[p,1] / (dh))
        j = np.floor(fj)
        hy = fj - j 
        
        q[i,j] = q[i,j] + (1-(hx)) * (1-(hy))
        q[i+1, j] = q[i+1, j] + (hx) * (1-(hy))
        q[i, j+1] = q[i, j+1] + (1-(hx)) * (hx) 
        q[i+1, j+1] = q[i+1, j+1] + (hx) * (hx) 

    rho = (sw + q_mp * q) / (dh * dh)
       
    rho[0,:] = 2 * rho[0,:]
    rho[-1, :] = 2 * rho[-1, :]
    rho[:, 0] = 2 * rho[:, 0]
    rho[:, -1] = 2 * rho[:, -1]
    
    rho = rho + (1 * 10 ** 4)
    #print(rho)
    ##on the order of +1E10

    #Calling potential solver 
    
    V = solver_2d(rho, tol, Te, n0, V, Q, M)
    print(V)
    ##START HERE!!! => nan nan nan... Check solver_2d 
    
    
    #E field solver
    
    #internal nodes 
    Ex[1:nx-1, :] = V[0:nx-2,:] - V[nx-(nx-2):, :]
    Ey[0: ,1:nx-1] = V[:, 0:ny-2] - V[:, 2:ny]

    #boundaries
    ##multiplied by 2 to keep values equivalent to internal nodes
    Ex[0,:] = 2* (V[0,:] - V[1,:]) 
    Ex[nx-1, :] = 2 * (V[nx-2,:] - V[nx-1, :])
    
    Ey[:, 0] = 2 * (V[:,0] - V[:,1])
    Ey[:,ny-1] = 2 * (V[:, ny-2] - V[:, ny-1])
    
    Ex = np.floor (Ex / (2 * dx))                                       
    Ey = Ey / (2 * dy)
    
    # Generate particles 
    
    if NP + np_in >= N:
        np_in = N - NP
   
    # Insert paritcles 
    
    p_pos[NP:NP+np_in, 1:] = np.random.rand(np_in,1) * dh 
    ##x position 
    
    p_pos[NP:NP+np_in, 1:] = np.random.rand(np_in,1) * Ly 
    ##y position 
    
    # Sample Maxwellian in x,y 
    # Add drift velocity in x 
    
    p_velo[NP:NP+np_in, 1:] = v_drift + (-1.5 + np.random.rand(np_in,1) + np.random.rand(np_in,1) 
                                         + np.random.rand(np_in, 1)) * vth 
    
    p_velo[NP:NP+np_in, 2:] = 0.5 * (-1.5 + np.random.rand(np_in,1) + np.random.rand(np_in,1) 
                                     + np.random.rand(np_in, 1)) * vth
    
    NP = NP + np_in
    
    # Move particles 
    p = 1 
    
    while p <= NP:
        # print('The number of particles is:', p)    
        
        fi = 1 + (p_pos[p,0]/dx)
        i = np.floor(fi)
        hx = fi - i  
                
        fj = 1+ (p_pos[p,1]/dy)
        j = np.floor(fj)
        hy = fj - j 
        
        E = np.zeros((nx,ny))
        E = ([Ex[i, j], Ey[i,j]]) * (1-(hx)) * (1-(hy))                                                
        E = E + ([Ex[i + 1, j], Ey[i + 1, j]]) * (hx) * (1-(hy))                                                
        E = E + ([Ex[i, j + 1], Ey[i + 1, j]]) * (1-(hx)) * (hy)                                                
        E = E + ([Ex[i + 1, j + 1], Ey[i + 1, j + 1]]) * (hx) * (hy)  
                
        F = QE * E 
        #print(F)
        
        a = F/MI
        
        p_pos[p, :] = p_velo[p, :] + a * dt 
        p_velo[p, :] = p_pos[p, :] + p_velo[p, :] * dt 
        
        if p_pos[p,1] < 0: 
            p_pos[p,1] = -p_pos[p,1]
            p_velo[p,1] = -p_velo[p,1]
            
        p = p + 1 
        
    #print(E) 
    ##E's are all 0...
    
    # Plot    
    ## Phase space plots work, contour does not  
    
#    if count%10 == 0 or count == N_ts:
#        fig = plt.figure(1, figsize=(8.0,6.0))
#        plt.contourf(V)
        #        plt.plot(p_pos, p_velo,'m.', ms=3)
#        #plt.title('Two-Stream Instability', fontsize = 16, fontweight = 'bold', color = 'w')
#        #plt.xticks([0, L], color = 'k', fontsize = 18)  # color = 'w'
#        plt.xlabel('x', fontsize = 18 , color = 'k')
#        #plt.yticks([-2*v0, -v0, 0, v0, 2*v0], color = 'k', fontsize = 18)
#        plt.ylabel('v', fontsize = 18, rotation=0 , color = 'k')
#        #plt.savefig('pngs/20180428/Run3_pngs_20180428/Frame_{count}.png'. format(count=count), frameon= True, transparent=True) #facecolor = 'k',
#        plt.show()
#        #plt.close()
   
print("Clocking in at %s seconds"  % (time.clock() - start))

Calculating...
[[ -1.30556092e+02   1.78335215e+04   2.39923472e+06   3.22798999e+08
    4.34301969e+10   5.84320898e+12   7.86160175e+14   1.05771986e+17
    1.42308316e+19  -1.30556092e+02]
 [ -2.63105408e+02   1.79676081e+04   6.04300333e+05   8.08508246e+07
    1.08777619e+10   1.46352169e+12   1.96905924e+14   2.64922231e+16
    3.56433096e+18              inf]
 [             inf              nan              nan              nan
               nan              nan              nan              nan
               nan              nan]
 [             nan              nan              nan              nan
               nan              nan              nan              nan
               nan              nan]
 [             nan              nan              nan              nan
               nan              nan              nan              nan
               nan              nan]
 [             nan              nan              nan              nan
               nan            

/Users/Isabel/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:50: RuntimeWarning: divide by zero encountered in true_divide
/Users/Isabel/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:55: RuntimeWarning: invalid value encountered in multiply
/Users/Isabel/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:144: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Isabel/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:145: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Isabel/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:146: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Isabel/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:147: VisibleDeprecationWarning: using a non-integer number instead of an integer will

TypeError: can't multiply sequence by non-int of type 'float'

### References 

1. https://www.particleincell.com/2010/es-pic-method/#charge_density